# TP2: Planificación de Conexión de Oficinas de Atención a Internet

Integrantes: Micaela Oliva, Camila Bernardez, Sol Valeggia, Juan Castore

## Contexto
Estamos en el área de planificación de una empresa de cobros minoristas, y tenemos la responsabilidad de conectar a Internet 56 oficinas de atención al cliente, distribuidas en el micro y macrocentro de la ciudad. La empresa de telefonía nos ha propuesto hasta 10 centrales operativas que pueden ser utilizadas para abastecer estas oficinas.


### 1. Modelo de Programación Lineal Entera

Planteamos el siguiente modelo para resolver la función objetivo, que consiste en minimizar el costo total de operación, decidiendo que centrales utilizar y con qué oficinas conectarlas, sujeto a ciertas restricciones:

  - **Índices**
    - $\mathbb{I} = \{1, \dots, 56\}$: Conjunto de oficinas.
    - $\mathbb{J} = \{1, \dots, 10\}$: Conjunto de centrales operativas.

  - **Parámetros**
    - $\mathrm{dist}_{ij}$: Distancia entre oficina $i$ y central $j$ $\text{ } \text{ } \forall i \in \mathbb{I}, \forall j \in \mathbb{J}$
    - $\mathrm{f} = 5700 \text{ } \mathrm{USD}$: costo fijo por abrir una central.
    - $\mathrm{demanda}_i$: Demanda de operaciones por hora de oficina $i$.
    - $\mathrm{capacidad}_j = 15000$: Capacidad máxima de operaciones por hora que puede atender la central $j$.
    - $cable = 0.017 \text{ } \mathrm{USD/m}$: Costo por metro de cable.

  - **Variables de Decisión**: 
     - $x_{ij}$: Variable binaria que indica si la oficina $i$ está conectada a la central $j$.
     - $y_j$: Variable binaria que indica si la central $j$ está en operación.

En base a lo definido anteriormente, la **función objetivo** a minimizar está dada por:

$$\mathrm{min} \sum_{j \in \mathbb{J}} f \cdot y_j \text{ } + \text{ } \sum_{i \in \mathbb{I}} \sum_{j \in \mathbb{J}} \text{} (\mathrm{dist}_{ij} \cdot cable) \cdot x_{ij}$$

Sujeto a las siguientes **restricciones**:

- Cada oficina está conectada a exactamente una central: $\sum_{j \in \mathbb{J}} x_{ij}=1 \text{ } \text{ } \forall i \in \mathbb{I}$

- La suma de demanda de las oficinas asociadas a una central no puede superar la capacidad de la misma: $\sum_{i \in \mathbb{I}} \mathrm{demanda}_i \cdot x_{ij} \leq \mathrm{capacidad}_j \cdot y_j  \text{ } \text{ } \forall j \in \mathbb{J}$

- Una oficina solo puede conectarse a una central si esa central está operativa: $x_{ij} \leq y_j \text{ } \text{ } \forall i \in \mathbb{I}, \forall j \in \mathbb{J}$

- Las variables binarias toman valor 0 o 1:
    - $x_{ij} \in \{0,1\}$ $\text{ } \text{ } \forall i \in \mathbb{I}, j \in \mathbb{J}$
    - $y_{ij} \in \{0,1\}$ $\text{ } \text{ } \forall j \in \mathbb{J}$

### 2. Análisis de Restricciones Adicionales
Una nueva restricción en el modelo que establece que el número de oficinas conectadas a cada central no puede superar las 10 puede plantearse como $\sum_{i \in \mathbb{I}}  x_{ij} \leq 10 \cdot y_j  \text{ } \text{ } \forall j \in \mathbb{J}$.

Sin embargo, no tienen ningún impacto sobre la función objetivo en sí: esta sigue siendo minimizar el costo total de operaciones, y se mantiene definida de la misma manera. Lo que podría cambiar es el óptimo, por ejemplo si se requiere aumentar la distancia promedio de las conexiones


### 3. Determinación de la Capacidad Mínima
La capacidad mínima necesaria por central debe ser igual a la demanda máxima de todas las oficinas si queremos garantizar la factibilidad del problema, considerando el planteo inicial donde no había límites a la cantidad de oficinas que se podían conectar a cada central. Es decir, $\mathrm{capacidad}_j \geq \sum_{i \in I}\mathrm{demanda}_i$.

En el caso de mantener la restricción de $\sum_{i \in \mathbb{I}}  x_{ij} \leq 10 \cdot y_j  \text{ } \text{ } \forall j \in \mathbb{J}$, la capacidad mínima sería la suma de las 10 mayores demandas, ya que en el peor caso, una central podría tener que atender a las 10 oficinas con mayor demanda. Cualquier otra combinación de oficinas resultaría en una demanda menor que se podría atender con esta capacidad.


### 4. Evaluación del Tiempo de Resolución


In [2]:
#Descargo datos
import pandas as pd
import numpy as np
import time
from ortools.linear_solver import pywraplp

#Llamamos al solver
solver = pywraplp.Solver.CreateSolver('SCIP') # Usamos SCIP

#Parámetros

# Leer el archivo con separación por espacio
C = pd.read_csv("centrales.txt", sep=" ", header = None)
C.columns = ['id', 'lat', 'long']
D = pd.read_csv("distancias.txt", sep="\t", header = None)
D = D.dropna(axis = 1, how='all')
D.columns = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
O = pd.read_csv("oficinas.txt", sep=" ", header = None)
O.columns = ['id', 'demanda', 'lat', 'long']


CAP = 15000 #Capacidad max que tiene cada central
F = 5700    #Costo fijo por abrir una central
CABLE = 0.017 #Costo por metro de cable

#Indices
I = list(range(len(O))) #Indices de oficinas
J = list(range(len(C))) #Indices de centrales

#Variables

y = {}  # y[j] = 1 si ela central j está abierta

for j in J:
    y[j] = solver.BoolVar(f'y[{j}]')

x = {}  # x[i, j] = 1 si la oficina i se conecta a la central j

for i in I:
    for j in J:
        x[i, j] = solver.BoolVar(f'x[{i},{j}]')

#Restricciones

#1. Cada oficina solo puede estar conectada a una central
for i in I:
    solver.Add(sum(x[i, j] for j in J) == 1)  

#2. La demanda de las oficinas conectadas debe ser menor a la capacidad total de la central
for j in J:
    solver.Add(sum(O['demanda'][i] * x[i, j] for i in I) <= CAP * y[j])

#3. Una central solo debe estar abierta si tiene una oficina conectada
for i in I:
    for j in J:
        solver.Add(x[i,j] <= y[j])

#4. Binarias
#Ya implemetado que las variables x e y sean binarias

#Función objetivo

#Sumatoria de costo total de centrales abiertas
costo_centrales = solver.Sum(F * y[j] for j in J)

#Doble Sumatoria de costo de conexion entre centrales y oficinas
costo_distancias = solver.Sum(D[f"c{j}"][i] * CABLE * x[i,j] for i in I for j in J)

solver.Minimize(costo_centrales + costo_distancias)

# Resolver
start_time = time.time()
status = solver.Solve()
end_time = time.time()

# Verificar resultado
if status == pywraplp.Solver.OPTIMAL:
    print("Solución óptima encontrada.")
    print("Valores de las variables y (centrales abiertas):")
    for j in J:
        print(f"Central {j} abierta: {y[j].solution_value()}")
    
    print("\nValores de las variables x (conexiones):")
    for j in J:
        for i in I:
            if x[i, j].solution_value() == 1:
                print(f"Oficina {i} conectada a Central {j}")
    
    print("\nValor óptimo de la función objetivo (costo total):", solver.Objective().Value())
    print(f"\nTiempo de resolución: {end_time - start_time:.4f} segundos")
else:
    print("No se encontró solución óptima.")

Solución óptima encontrada.
Valores de las variables y (centrales abiertas):
Central 0 abierta: 1.0
Central 1 abierta: 1.0
Central 2 abierta: 1.0
Central 3 abierta: 0.0
Central 4 abierta: 1.0
Central 5 abierta: 0.0
Central 6 abierta: 1.0
Central 7 abierta: 0.0
Central 8 abierta: 1.0
Central 9 abierta: 0.0

Valores de las variables x (conexiones):
Oficina 14 conectada a Central 0
Oficina 45 conectada a Central 0
Oficina 46 conectada a Central 0
Oficina 47 conectada a Central 0
Oficina 48 conectada a Central 0
Oficina 49 conectada a Central 0
Oficina 50 conectada a Central 0
Oficina 51 conectada a Central 0
Oficina 52 conectada a Central 0
Oficina 53 conectada a Central 0
Oficina 0 conectada a Central 1
Oficina 1 conectada a Central 1
Oficina 2 conectada a Central 1
Oficina 13 conectada a Central 1
Oficina 15 conectada a Central 1
Oficina 28 conectada a Central 1
Oficina 29 conectada a Central 1
Oficina 41 conectada a Central 1
Oficina 42 conectada a Central 1
Oficina 43 conectada a Cent

In [7]:
#Descargo datos
import pandas as pd
import numpy as np
import time
from ortools.linear_solver import pywraplp

#Llamamos al solver
solver = pywraplp.Solver.CreateSolver('SCIP') # Usamos SCIP

#Parámetros

# Leer el archivo con separación por espacio
C = pd.read_csv("centrales.txt", sep=" ", header = None)
C.columns = ['id', 'lat', 'long']
D = pd.read_csv("distancias.txt", sep="\t", header = None)
D = D.dropna(axis = 1, how='all')
D.columns = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
O = pd.read_csv("oficinas.txt", sep=" ", header = None)
O.columns = ['id', 'demanda', 'lat', 'long']


CAP = 15000 #Capacidad max que tiene cada central
F = 5700    #Costo fijo por abrir una central
CABLE = 0.017 #Costo por metro de cable

#Indices
I = list(range(len(O))) #Indices de oficinas
J = list(range(len(C))) #Indices de centrales

#Variables

y = {}  # y[j] = 1 si ela central j está abierta

for j in J:
    y[j] = solver.BoolVar(f'y[{j}]')

x = {}  # x[i, j] = 1 si la oficina i se conecta a la central j

for i in I:
    for j in J:
        x[i, j] = solver.BoolVar(f'x[{i},{j}]')

#Restricciones

#1. Cada oficina solo puede estar conectada a una central
for i in I:
    solver.Add(sum(x[i, j] for j in J) == 1)  

#2. La demanda de las oficinas conectadas debe ser menor a la capacidad total de la central
for j in J:
    solver.Add(sum(O['demanda'][i] * x[i, j] for i in I) <= CAP * y[j])

#3. Una central solo debe estar abierta si tiene una oficina conectada
for i in I:
    for j in J:
        solver.Add(x[i,j] <= y[j])

#4. Binarias
#Ya implemetado que las variables x e y sean binarias

#5. Opcional: Que cada central tenga como maximo 10 oficinas conectadas 
for j in J:
    solver.Add(sum(x[i,j] for i in I) <= 10*y[j])


#Función objetivo

#Sumatoria de costo total de centrales abiertas
costo_centrales = solver.Sum(F * y[j] for j in J)

#Doble Sumatoria de costo de conexion entre centrales y oficinas
costo_distancias = solver.Sum(D[f"c{j}"][i] * CABLE * x[i,j] for i in I for j in J)

solver.Minimize(costo_centrales + costo_distancias)

# Resolver
start_time = time.time()
status = solver.Solve()
end_time = time.time()

# Verificar resultado
if status == pywraplp.Solver.OPTIMAL:
    print("Solución óptima encontrada.")
    print("Valores de las variables y (centrales abiertas):")
    for j in J:
        print(f"Central {j} abierta: {y[j].solution_value()}")
    
    print("\nValores de las variables x (conexiones):")
    for j in J:
        for i in I:
            if x[i, j].solution_value() == 1:
                print(f"Oficina {i} conectada a Central {j}")
    
    print("\nValor óptimo de la función objetivo (costo total):", solver.Objective().Value())
    print(f"\nTiempo de resolución: {end_time - start_time:.4f} segundos")
else:
    print("No se encontró solución óptima.")

Solución óptima encontrada.
Valores de las variables y (centrales abiertas):
Central 0 abierta: 1.0
Central 1 abierta: 1.0
Central 2 abierta: 1.0
Central 3 abierta: 0.0
Central 4 abierta: 1.0
Central 5 abierta: 0.0
Central 6 abierta: 1.0
Central 7 abierta: 0.0
Central 8 abierta: 1.0
Central 9 abierta: 0.0

Valores de las variables x (conexiones):
Oficina 14 conectada a Central 0
Oficina 45 conectada a Central 0
Oficina 46 conectada a Central 0
Oficina 47 conectada a Central 0
Oficina 48 conectada a Central 0
Oficina 49 conectada a Central 0
Oficina 50 conectada a Central 0
Oficina 51 conectada a Central 0
Oficina 52 conectada a Central 0
Oficina 53 conectada a Central 0
Oficina 0 conectada a Central 1
Oficina 1 conectada a Central 1
Oficina 2 conectada a Central 1
Oficina 13 conectada a Central 1
Oficina 15 conectada a Central 1
Oficina 28 conectada a Central 1
Oficina 29 conectada a Central 1
Oficina 41 conectada a Central 1
Oficina 42 conectada a Central 1
Oficina 43 conectada a Cent

## Conclusiones
Este análisis permitirá tomar decisiones informadas sobre la selección y configuración de las centrales operativas y la conexión de las oficinas, optimizando tanto el costo como la eficiencia operativa en el abastecimiento de las oficinas con servicio de Internet.
